In [1]:
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [8]:
cwd = os.getcwd()
PROJECT_PATH = cwd.split("quince_assignment")[0] + 'quince_assignment'
sys.path.insert(0, PROJECT_PATH)

In [9]:
event_data = pd.read_csv(
    f"{PROJECT_PATH}/src/data/event_202012182112.csv.gz", compression="gzip")

In [10]:
event_data['event_time'] = pd.to_datetime(event_data['event_time'],
                                          format="%Y-%m-%d %H:%M:%S")

In [11]:
event_data = event_data.sort_values(['lb_user_id', 'session_id', 'event_time']).reset_index(drop=True)

In [15]:
unique_items = event_data.nunique()

In [17]:
unique_items.to_dict()

{'event_id': 3638376,
 'event_type': 12,
 'session_id': 817575,
 'event_time': 2593479,
 'lb_user_id': 588808,
 'email': 62877,
 'product_id': 266,
 'product_category': 5,
 'product_type': 28,
 'order_id': 32430,
 'collection_name': 176,
 'ip': 419149,
 'device': 3,
 'url': 830709,
 'device_category': 10,
 'user_agent': 61563,
 'user_gender': 2,
 'user_state': 356,
 'traffic_source': 0}

In [21]:
event_type_order = {
    'checkout_page_view_contact_information':
    '4a. checkout_page_view_contact_information',
    'email_subscription_view': '6. email_subscription_view',
    'checkout_page_view_payment_method':
    '4c. checkout_page_view_payment_method',
    'checkout_page_view_stock_problems':
    '4e. checkout_page_view_stock_problems',
    'checkout_page_view_review': '4f. checkout_page_view_review',
    'checkout_page_view_processing': '4d. checkout_page_view_processing',
    'add_to_cart': '3. add_to_cart',
    'collection_view': '1. collection_view',
    'checkout_page_view_forward': '4g. checkout_page_view_forward',
    'product_view': '2. product_view',
    'checkout_page_view_shipping_method':
    '4b. checkout_page_view_shipping_method',
    'order_complete': '5. order_complete'
}

In [25]:
event_data.replace({'event_type': event_type_order}, inplace=True)

In [28]:
event_data['event_type'].value_counts(normalize=True).sort_index()

1. collection_view                            0.362388
2. product_view                               0.375208
3. add_to_cart                                0.053908
4a. checkout_page_view_contact_information    0.020481
4b. checkout_page_view_shipping_method        0.008427
4c. checkout_page_view_payment_method         0.008819
4d. checkout_page_view_processing             0.009373
4e. checkout_page_view_stock_problems         0.000858
4f. checkout_page_view_review                 0.000053
4g. checkout_page_view_forward                0.000197
5. order_complete                             0.009119
6. email_subscription_view                    0.151167
Name: event_type, dtype: float64

In [13]:
event_data[event_data['lb_user_id']=='_0000slud8']

,event_id,event_type,session_id,event_time,lb_user_id,email,product_id,product_category,product_type,order_id,collection_name,ip,device,url,device_category,user_agent,user_gender,user_state,traffic_source
0,571444b9-d794-477b-8dd7-d60868307bc9,product_view,_rh51mudfr,2020-09-22 14:21:01,_0000slud8,NaN,4.371588e+12,Home,Bedding,NaN,NaN,98.117.41.216,desktop,https://www.onequince.com/home/sheets/french-l...,Windows,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,NaN,NaN,NaN
1,e7c4ada6-3e21-4059-74ec-2274dc92101d,collection_view,_rh51mudfr,2020-09-22 14:21:54,_0000slud8,NaN,NaN,NaN,NaN,NaN,New Arrivals,98.117.41.216,desktop,https://www.onequince.com/collections/new-arri...,Windows,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,NaN,NaN,NaN
2,579535d3-8969-4109-fcaa-4040175487f5,product_view,_rh51mudfr,2020-09-22 14:22:49,_0000slud8,NaN,4.351721e+12,Home,Bedding,NaN,NaN,98.117.41.216,desktop,https://www.onequince.com/home/sheets/french-l...,Windows,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,NaN,NaN,NaN


In [12]:
event_data

,event_id,event_type,session_id,event_time,lb_user_id,email,product_id,product_category,product_type,order_id,collection_name,ip,device,url,device_category,user_agent,user_gender,user_state,traffic_source
0,571444b9-d794-477b-8dd7-d60868307bc9,product_view,_rh51mudfr,2020-09-22 14:21:01,_0000slud8,NaN,4.371588e+12,Home,Bedding,NaN,NaN,98.117.41.216,desktop,https://www.onequince.com/home/sheets/french-l...,Windows,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,NaN,NaN,NaN
1,e7c4ada6-3e21-4059-74ec-2274dc92101d,collection_view,_rh51mudfr,2020-09-22 14:21:54,_0000slud8,NaN,NaN,NaN,NaN,NaN,New Arrivals,98.117.41.216,desktop,https://www.onequince.com/collections/new-arri...,Windows,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,NaN,NaN,NaN
2,579535d3-8969-4109-fcaa-4040175487f5,product_view,_rh51mudfr,2020-09-22 14:22:49,_0000slud8,NaN,4.351721e+12,Home,Bedding,NaN,NaN,98.117.41.216,desktop,https://www.onequince.com/home/sheets/french-l...,Windows,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,NaN,NaN,NaN
3,26514163-7313-4b71-d116-71c882d09193,collection_view,_8oaz8a9lb,2020-10-27 04:17:14,_0006tbdmt,NaN,NaN,NaN,NaN,NaN,New Arrivals,71.237.88.14,desktop,https://www.onequince.com/collections/new-arri...,macOS,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,NaN,NaN,NaN
4,de54b116-c7ed-4116-2e79-bcacb62569a8,email_subscription_view,_8oaz8a9lb,2020-10-27 04:17:23,_0006tbdmt,NaN,NaN,NaN,NaN,NaN,NaN,71.237.88.14,desktop,https://www.onequince.com/collections/new-arri...,macOS,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3638371,583c5c7a-813b-478b-6823-218fdf11d6e0,email_subscription_view,NaN,2020-11-30 23:41:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.70.176.249,mobile,https://www.onequince.com/men/cashmere?utm_sou...,macOS,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,NaN,NaN,NaN
3638372,b1fbb1af-1611-448c-75e4-045bfad27fcf,collection_view,NaN,2020-11-30 23:41:38,NaN,NaN,NaN,NaN,NaN,NaN,Cashmere,73.70.176.249,mobile,https://www.onequince.com/men/cashmere?utm_sou...,macOS,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,NaN,NaN,NaN
3638373,4701456b-f0a7-4de7-de1a-3d8b331fe618,email_subscription_view,NaN,2020-11-30 23:41:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.70.176.249,mobile,https://www.onequince.com/men/cashmere?utm_sou...,macOS,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,NaN,NaN,NaN
3638374,670220a6-50ee-4eeb-efa8-65d80eee6c37,email_subscription_view,NaN,2020-11-30 23:44:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.70.176.249,mobile,https://www.onequince.com/?utm_source=pepperja...,macOS,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,NaN,NaN,NaN


In [ ]:
print(f"Event data has {event_data.shape[0]} rows and {event_data.shape[1]} columns")

In [ ]:
event_data['event_type'].value_counts()

In [ ]:
event_data[event_data['lb_user_id']=='_0006tbdmt']